In [6]:
import os
import time
import geopandas as gpd
import pandas as pd
import numpy as np
#from datetime import datetime
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm_notebook as tqdm
from threading import Thread
from shapely.geometry import LineString
from shapely.geometry import Point

In [1]:
# Разбивает датафрейм на части
def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [2]:
# Расчёт растояния между координатами
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 * 1000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [3]:
def calc_distance(track_data, stop_data, mr_id, delta_distance = 100):
    time_ = time.time()
    time.sleep(36)
    # Формируем доп.столбцы в Dataframe, где указываем координаты посл.точки в треке, а затем переводим в объект geopandas со стобцом геометрии 'LineString' (для удобного отображения на карте)
    print('calc_distance mr_id {0} : geometry'.format(mr_id))
    track_data['rd_long_to'] = track_data['long']
    track_data['rd_lat_to'] = track_data['lat']
    track_data = gpd.GeoDataFrame(track_data, 
                     geometry = [LineString(xy) for xy in zip([Point(xy) for xy in zip(track_data['long'], track_data['lat'])], 
                                                              [Point(xy) for xy in zip(track_data['rd_long_to'], track_data['rd_lat_to'])])])
    
    # Объединяем таблицу с треками с таблицей последовательности остановок (каждую точку трека с каждой остановкой в рамках одного маршрута)
    print('calc_distance mr_id {0} : merge'.format(mr_id))
    df_cartesian = stop_data.merge(track_data, on = 'mr_id', how = 'outer')
    df_cartesian = df_cartesian[['mr_id','rl_id','rc_orderby','ssId','tr_id','timenav','geometry','lat','long','ss_lat','ss_long','tabelnum']]
    df_cartesian['len'] = 0
    
    # Запускаем расчёт расстояний между всеми точками трека
    df_len = pd.DataFrame()
    #split_list = split_dataframe(df_cartesian, 100000)
    i = 0
    # на какое количество частей разбиваем таблицу
    cnt = 1000
    print('calc_distance mr_id {0} : haversine'.format(mr_id))
    for df in np.array_split(df_cartesian, cnt):
        print('calc_distance mr_id {0} : current {1}...{2}'.format(mr_id, i, cnt))
        df['len'] = df.apply(lambda row: haversine(row.long, row.lat, row.ss_long, row.ss_lat), axis = 1)
        df_len = df_len.append(df, ignore_index=True)
        i += 1
        
    df_cartesian = df_len
    del df_len
    # Сохраняем csv для перехода расчётов в Perl c нужными нам полями:
    # mr_id - идентификатор маршрута,
    # rl_id - идентификатор направления, на котором сидит остановка Х
    # rc_orderby - порядковый номер остановки Х на направлении rl_id маршрута mr_id
    # ssId - уникальный идентификатор остановки Х
    # tr_id - уникальный идентификатор транспортного средства
    # timenav - время записи точки трека
    # len - расстояние от точки трека до остановки Х
    # geometry - поле Linestring для отрисовки трека
    # ss_lat - широта точки трека
    # ss_long - долгота точки трека
    # tabelnum - табельный номер
    # number_circle - посл. номер поездки от одной конечки до другой на конкретном tr_id (сейчас везде =1)
    df_cartesian = df_cartesian[['mr_id','rl_id','rc_orderby','ssId','tr_id','timenav','len','geometry','ss_lat','ss_long','tabelnum']]
    df_cartesian['number_circle'] = 1
    df_cartesian_sort = df_cartesian[df_cartesian['len'] < delta_distance].sort_values(by=['mr_id','tr_id','timenav','len'])
    df_cartesian_sort.to_csv('for_perl_mr_id_{0}_2019_10_17.csv'.format(str(mr_id)), sep=";", header = True, index = False)
    
    end_time = time.time()

    if end_time - time_ < 1:
        time.sleep(2)
    return 1

In [4]:
def merge___(path):
    """объединение всех файликов"""
    files = [f for f in os.listdir(path) if 'for_perl_mr_id_' in f]
    with open(path + 'for_perl.csv', 'a') as f:
        for file in files:
            try:
                with open(path + file, 'r') as t:
                    f.write(t.read())
            except:
                continue
    print('объединение закончено')
    return

In [7]:
path = './data/oct/16/'
# Cчитываем файлы, содержащие треки за какое-то одно число
files = [f for f in os.listdir(path) if '_4.csv' in f]

df_4 = pd.DataFrame()
for file in files:
    print(file)
    m = pd.read_csv(path + file, sep = '\t', encoding='windows-1251')
    df_4 = df_4.append(m, ignore_index = True)
    del m

2019-10-16_s2_4.csv


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2019-10-16_s3_4.csv
2019-10-16_s5_4.csv
2019-10-16_s6_4.csv
2019-10-16_s7_4.csv


In [9]:
# Считываем файл с последовательным списком остановок на маршрутах ('stop_seq.csv')
stop_seq = pd.read_csv(path + 'stop_seq.csv', sep = ';', encoding = 'utf8')
stop_seq = stop_seq.sort_values(['mr_id', 'rl_id', 'rc_orderby']).drop_duplicates().reset_index(drop = True)

In [ ]:

# Список маршуртов, для которых есть валидации
#Samara_mr_id_marsh_valid_id = pd.read_csv(path + 'Samara_mr_id_marsh_valid_id_ for_research_2.csv', sep = ';', encoding = 'utf8')

#marsh = Samara_mr_id_marsh_valid_id['mr_id'].unique()
#print(marsh)
marsh = stop_seq['mr_id'].unique()

# =============================================================================
# Вызов с потоками
# =============================================================================
thread = 0
for mr_id in marsh:
    print(mr_id)
    # расчет порциями по 10 потоков
#    if (thread >= 0) and (thread < 10):
    if (thread >= 0) and (thread < 100):
        if __name__ == '__main__':
            time.sleep(3)
            df_4_small = df_4[df_4['mr_id'] == mr_id].reset_index(drop=True)
            stop_seq_small = stop_seq[stop_seq['mr_id'] == mr_id].reset_index(drop=True)
            
            process = Thread(target=calc_distance, args=[df_4_small, stop_seq_small, mr_id])
            process.start()
    thread += 1